# arXiv Bilingual Reader - TranslateGemma

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jimmyliao/trans-gemma/blob/main/arxiv-reader-colab.ipynb)

**專注於雙語閱讀學習的 arXiv 論文翻譯工具**

## 🎯 核心特色

- 📖 **雙語對照閱讀**：原文與翻譯並列，學習英文學術寫作
- 🧠 **智慧混合模式**：純文字頁面快速翻譯，含圖表頁面保留視覺上下文
- 📚 **分章節翻譯**：按論文結構（Abstract, Method, Experiments）組織
- 💡 **術語對照表**：自動提取專業術語，建立中英對照
- 🇹🇼 **繁體中文優化**：確保輸出台灣慣用的繁體中文（已修復官方 bug）

## 🎓 適合誰使用？

- ✅ 研究生學習閱讀英文論文
- ✅ 研究者深入研讀重要文獻
- ✅ 準備論文寫作/審稿的學者
- ✅ 想學習英文學術寫作的人

## ⚡ 快速開始

1. 執行 Cell 1-2（環境設置 + 認證）
2. 在 Cell 3 輸入 arXiv 論文 ID 和設定
3. 執行 Cell 4 開始翻譯
4. 享受雙語對照閱讀！

---

## 👤 About the Author

**Jimmy Liao** - AI Google Developer Expert (GDE), CTO/Co-Founder

- 🐦 Twitter: [@jimmyliao](https://twitter.com/jimmyliao)
- 💼 LinkedIn: [jimmyliao](https://linkedin.com/in/jimmyliao)
- 📝 Blog: [memo.jimmyliao.net](https://memo.jimmyliao.net)
- 🔗 GitHub: [jimmyliao/trans-gemma](https://github.com/jimmyliao/trans-gemma)

---

## 1️⃣ Setup: Clone Repository & Install Dependencies

In [ ]:
# Clone repository (single source of truth)
!rm -rf trans-gemma
!git clone https://github.com/jimmyliao/trans-gemma.git
%cd trans-gemma

# Install dependencies
!pip install uv -q
!uv pip install --system -e ".[examples]"

## 2️⃣ HuggingFace Authentication

**IMPORTANT:** TranslateGemma is a gated model. You need to:
1. Get a HuggingFace token from [https://huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)
2. Accept model access at [https://huggingface.co/google/translategemma-4b-it](https://huggingface.co/google/translategemma-4b-it)

### 🔐 Configuration Methods (Choose ONE based on your environment)

#### **Option A: Web Colab** (Using browser)
1. Click the 🔑 icon on left sidebar
2. Add secret: `HF_TOKEN` = your token
3. Run the cell below → Token loaded automatically from Colab Secrets

#### **Option B: VS Code Colab Extension** (Using VS Code locally)

**⚠️ Important:** Your local `.env` file is NOT automatically synced to remote Colab runtime!

**Solution: Create .env in remote runtime**

The cell below will:
1. First check if `.env` exists in remote runtime
2. If not found, prompt you to enter token
3. Automatically create `.env` file in remote runtime
4. Use this token for authentication

This way, you only need to enter your token once per runtime session.

#### **Option C: Manual Input Every Time** (Not recommended)
Skip .env creation and enter token manually each time.

---

**What happens when you run the cell below?**
1. Checks for `.env` in current directory (remote runtime)
2. If not found, prompts for token and creates `.env`
3. If found, reads token from `.env`
4. Authenticates with HuggingFace


In [ ]:
from huggingface_hub import login
import os
from pathlib import Path

def get_hf_token():
    """Smart HF Token retrieval with .env creation for VS Code"""
    
    # Method 1: Try .env file in current directory
    env_file = Path('.env')
    
    if env_file.exists():
        try:
            with open('.env', 'r') as f:
                for line in f:
                    line = line.strip()
                    if line.startswith('HF_TOKEN='):
                        token = line.split('=', 1)[1].strip().strip('"').strip("'")
                        if token:
                            print("✅ HF_TOKEN loaded from .env file")
                            return token
            print("⚠️  .env file found but HF_TOKEN not set correctly")
        except Exception as e:
            print(f"⚠️  Error reading .env: {e}")
    
    # Method 2: Try environment variables
    token = os.getenv('HF_TOKEN') or os.getenv('HUGGING_FACE_HUB_TOKEN')
    if token:
        print("✅ HF_TOKEN loaded from environment variable")
        return token
    
    # Method 3: Try Colab Secrets (Web Colab only)
    try:
        from google.colab import userdata
        token = userdata.get('HF_TOKEN')
        print("✅ HF_TOKEN loaded from Colab Secrets (Web Colab)")
        return token
    except Exception:
        pass
    
    # Method 4: Prompt for token and create .env (VS Code friendly)
    print("\n" + "="*80)
    print("⚠️  HF_TOKEN not found - Creating .env file")
    print("="*80)
    print("\n📝 Please enter your HuggingFace token:")
    print("   Get token: https://huggingface.co/settings/tokens")
    print("   Accept access: https://huggingface.co/google/translategemma-4b-it")
    print("\n💡 Your token will be saved to .env for this runtime session")
    print()
    
    token = input("HuggingFace Token: ").strip()
    
    if token:
        # Save to .env for future use in this session
        try:
            with open('.env', 'w') as f:
                f.write(f"HF_TOKEN={token}\n")
            print("\n✅ Token saved to .env file (runtime session)")
            print("   Next time you run this cell, it will load automatically")
        except Exception as e:
            print(f"\n⚠️  Could not save to .env: {e}")
            print("   Token will work this time but won't persist")
        
        return token
    else:
        raise ValueError("❌ HF_TOKEN is required to use TranslateGemma")

# Authenticate
try:
    HF_TOKEN = get_hf_token()
    os.environ['HF_TOKEN'] = HF_TOKEN
    login(token=HF_TOKEN)
    print("\n✅ Successfully authenticated with HuggingFace\n")
except Exception as e:
    print(f"\n❌ Authentication failed: {e}\n")
    raise

## 3️⃣ Configuration: Bilingual Reading Settings

**配置你的論文和翻譯設定**

In [ ]:
# ============================================
# arXiv Paper Configuration
# ============================================

# arXiv Paper ID (from URL: https://arxiv.org/abs/XXXX.XXXXX)
ARXIV_ID = "2601.09012v2"  # TranslateGemma Technical Report

# ============================================
# Bilingual Reading Settings
# ============================================

# Enable bilingual mode (always recommended for learning)
BILINGUAL_MODE = True

# Target language
TARGET_LANG = "zh-TW"  # Traditional Chinese
SOURCE_LANG = "en"     # English

# ============================================
# Translation Sections (Page Ranges)
# ============================================

# 🚀 FULL TRANSLATION MODE
# Define which sections to translate
# Format: "section_name": (start_page, end_page)
SECTIONS = {
    "abstract": (1, 1),      # Abstract (Page 1)
    "method": (3, 5),        # Method section (Pages 3-5)
    "experiments": (7, 9),   # Experiments (Pages 7-9)
}

# ============================================
# Display Settings
# ============================================

# Extract technical terms for glossary
EXTRACT_TERMS = True

# Display options
DISPLAY_MODE = "rich"  # Options: "simple", "rich", "html"
# - simple: Plain text (fast)
# - rich: Colored boxes with side-by-side layout (recommended)
# - html: HTML table (best for saving/sharing)

# Save output to file
SAVE_TO_FILE = True  # Save translation to HTML file

# Debug mode (set to True to see detailed progress)
DEBUG_MODE = True  # Enable detailed debug output

# ============================================
# Performance Note
# ============================================
# Using text-only mode for better quality and speed
# Text mode: ~20-25s/page
# All pages will be processed as pure text

# ============================================
# Summary
# ============================================

print("\n" + "="*80)
print("🎯 arXiv Bilingual Reader - Configuration")
print("="*80)
print(f"\n📄 Paper: arXiv:{ARXIV_ID}")
print(f"🌐 Translation: {SOURCE_LANG} → {TARGET_LANG}")
print(f"📖 Bilingual Mode: {'✅ Enabled' if BILINGUAL_MODE else '❌ Disabled'}")
print(f"📺 Display Mode: {DISPLAY_MODE}")
print(f"💾 Save to file: {'✅ Yes' if SAVE_TO_FILE else '❌ No'}")
print(f"\n📚 Sections to translate:")
total_pages = 0
for section, (start, end) in SECTIONS.items():
    page_count = end - start + 1
    total_pages += page_count
    pages = f"Page {start}" if start == end else f"Pages {start}-{end}"
    print(f"   • {section.capitalize()}: {pages} ({page_count} pages)")
print(f"\n🚀 FULL MODE: Translating {total_pages} pages")
print(f"⚡ Mode: Text-only (fast and accurate)")
print(f"   Estimated time: ~{int(total_pages * 23 / 60)} minutes {(total_pages * 23) % 60} seconds")
print(f"\n💡 Ready to translate! Execute Cell 4 to start.")
print("="*80 + "\n")

## 4️⃣ Translate: Smart Bilingual Mode

**執行智慧雙語翻譯**

- 📄 Text mode: 20-25s/page (pure text pages)
- 🖼️ Image mode: 40-50s/page (pages with figures/tables)

In [ ]:
import sys
import time
import gc
import torch
from pathlib import Path

# Add examples to path
sys.path.insert(0, 'examples')
sys.path.insert(0, 'examples/backends')

# Import backends
from transformers_backend import TransformersBackend

# Import utilities
import urllib.request
import fitz  # PyMuPDF
import re
from IPython.display import HTML, display

# ============================================
# Install and import tqdm for progress bar
# ============================================
try:
    from tqdm.auto import tqdm
except ImportError:
    !pip install tqdm -q
    from tqdm.auto import tqdm

# ============================================
# Helper Functions
# ============================================
def download_arxiv(arxiv_id):
    """Download arXiv PDF"""
    clean_id = arxiv_id.split('v')[0] if 'v' in arxiv_id else arxiv_id
    url = f"https://arxiv.org/pdf/{clean_id}.pdf"
    filename = f"{arxiv_id}.pdf"
    
    print(f"📥 Downloading from arXiv: {arxiv_id}")
    urllib.request.urlretrieve(url, filename)
    
    doc = fitz.open(filename)
    total_pages = len(doc)
    doc.close()
    
    print(f"✅ Downloaded: {filename} ({total_pages} pages)\n")
    return filename, total_pages

def extract_text_from_page(pdf_path, page_num):
    """Extract text from specific PDF page"""
    doc = fitz.open(pdf_path)
    page = doc[page_num - 1]
    text = page.get_text()
    doc.close()
    return text.strip()

def display_bilingual_simple(original, translation, page_num, section_name, time_taken=0):
    """Simple text display"""
    print("\n" + "="*80)
    print(f"📄 {section_name.upper()} - Page {page_num} (⏱️  {time_taken:.1f}s)")
    print("="*80)
    
    if BILINGUAL_MODE:
        print("\n" + "─"*80)
        print("原文 (Original)")
        print("─"*80)
        orig_display = original[:800] + "..." if len(original) > 800 else original
        print(orig_display)
        
        print("\n" + "─"*80)
        print("翻譯 (Translation)")
        print("─"*80)
        print(translation)
        print("\n" + "="*80)
    else:
        print(translation)
        print("\n" + "="*80)

def display_bilingual_rich(results, current_index=0):
    """Rich HTML display with navigation"""
    total_pages = len(results)
    result = results[current_index]
    
    orig_preview = result['original'][:800] + "..." if len(result['original']) > 800 else result['original']
    
    # Navigation buttons
    nav_buttons = ""
    if total_pages > 1:
        prev_disabled = "disabled" if current_index == 0 else ""
        next_disabled = "disabled" if current_index == total_pages - 1 else ""
        
        nav_buttons = f"""
        <div style="display: flex; justify-content: space-between; align-items: center; 
                    padding: 10px 20px; background: #f8f9fa; border-bottom: 2px solid #e0e0e0;">
            <button onclick="showPage({current_index - 1})" {prev_disabled}
                    style="padding: 8px 16px; background: #667eea; color: white; border: none; 
                           border-radius: 4px; cursor: pointer; font-size: 14px;" 
                    {'' if not prev_disabled else 'style="opacity: 0.5; cursor: not-allowed;"'}>
                ◀ Previous
            </button>
            <span style="color: #495057; font-weight: bold;">
                Page {result['page_num']} / {results[-1]['page_num']} 
                ({current_index + 1} of {total_pages})
            </span>
            <button onclick="showPage({current_index + 1})" {next_disabled}
                    style="padding: 8px 16px; background: #667eea; color: white; border: none; 
                           border-radius: 4px; cursor: pointer; font-size: 14px;"
                    {'' if not next_disabled else 'style="opacity: 0.5; cursor: not-allowed;"'}>
                Next ▶
            </button>
        </div>
        """
    
    html = f"""
    <div id="page-{current_index}" class="translation-page" 
         style="margin: 20px 0; border: 2px solid #e0e0e0; border-radius: 8px; overflow: hidden;">
        {nav_buttons}
        <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                    color: white; padding: 12px 20px; font-weight: bold;">
            📄 {result['section'].upper()} - Page {result['page_num']} 
            <span style="float: right; font-weight: normal;">⏱️ {result['time']:.1f}s</span>
        </div>
    """
    
    if BILINGUAL_MODE:
        html += f"""
        <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 0;">
            <div style="padding: 20px; border-right: 2px solid #e0e0e0; background: #f8f9fa;">
                <div style="color: #495057; font-weight: bold; margin-bottom: 10px; 
                           font-size: 14px;">原文 (Original)</div>
                <div style="color: #212529; line-height: 1.6; white-space: pre-wrap; 
                           font-family: 'Segoe UI', sans-serif;">{orig_preview}</div>
            </div>
            <div style="padding: 20px; background: #ffffff;">
                <div style="color: #495057; font-weight: bold; margin-bottom: 10px; 
                           font-size: 14px;">翻譯 (Translation)</div>
                <div style="color: #212529; line-height: 1.6; white-space: pre-wrap; 
                           font-family: 'Segoe UI', 'Microsoft JhengHei', sans-serif;">{result['translation']}</div>
            </div>
        </div>
        """
    else:
        html += f"""
        <div style="padding: 20px;">
            <div style="color: #212529; line-height: 1.6; white-space: pre-wrap; 
                       font-family: 'Segoe UI', 'Microsoft JhengHei', sans-serif;">{result['translation']}</div>
        </div>
        """
    
    html += "</div>"
    
    # Add keyboard navigation hint
    if total_pages > 1 and current_index == 0:
        html += """
        <div style="text-align: center; color: #666; font-size: 14px; margin-top: 10px;">
            💡 提示：可以使用鍵盤 ← → 方向鍵快速切換頁面（在下載的 HTML 檔案中）
        </div>
        """
    
    display(HTML(html))

def generate_interactive_html(results, arxiv_id, source_lang, target_lang, all_terms):
    """Generate interactive HTML with page navigation"""
    
    pages_html = ""
    for idx, result in enumerate(results):
        orig_preview = result['original'][:800] + "..." if len(result['original']) > 800 else result['original']
        
        display_style = "" if idx == 0 else "display: none;"
        
        page_html = f"""
        <div id="page-{idx}" class="translation-page" style="{display_style}">
            <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                        color: white; padding: 12px 20px; font-weight: bold;">
                📄 {result['section'].upper()} - Page {result['page_num']} 
                <span style="float: right; font-weight: normal;">⏱️ {result['time']:.1f}s</span>
            </div>
            <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 0;">
                <div style="padding: 20px; border-right: 2px solid #e0e0e0; background: #f8f9fa;">
                    <div style="color: #495057; font-weight: bold; margin-bottom: 10px; 
                               font-size: 14px;">原文 (Original)</div>
                    <div style="color: #212529; line-height: 1.6; white-space: pre-wrap; 
                               font-family: 'Segoe UI', sans-serif;">{orig_preview}</div>
                </div>
                <div style="padding: 20px; background: #ffffff;">
                    <div style="color: #495057; font-weight: bold; margin-bottom: 10px; 
                               font-size: 14px;">翻譯 (Translation)</div>
                    <div style="color: #212529; line-height: 1.6; white-space: pre-wrap; 
                               font-family: 'Segoe UI', 'Microsoft JhengHei', sans-serif;">{result['translation']}</div>
                </div>
            </div>
        </div>
        """
        pages_html += page_html
    
    # Terms glossary
    terms_html = ""
    if all_terms:
        terms_html = """
        <div style="margin-top: 40px; padding: 20px; background: white; 
                    border-radius: 8px; border: 2px solid #667eea;">
            <h2 style="color: #667eea;">📚 Technical Terms Glossary</h2>
            <table style="width: 100%; border-collapse: collapse;">
        """
        for en, zh in sorted(all_terms.items()):
            terms_html += f"""
            <tr style="border-bottom: 1px solid #eee;">
                <td style="padding: 8px; width: 50%;">{en}</td>
                <td style="padding: 8px; color: #667eea;">→ {zh}</td>
            </tr>
            """
        terms_html += "</table></div>"
    
    html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
        <title>TranslateGemma Translation - arXiv:{arxiv_id}</title>
        <style>
            body {{ 
                font-family: 'Segoe UI', 'Microsoft JhengHei', sans-serif; 
                max-width: 1200px; 
                margin: 40px auto; 
                padding: 0 20px; 
                background: #f5f5f5; 
            }}
            h1 {{ color: #667eea; text-align: center; margin-bottom: 40px; }}
            .timestamp {{ text-align: center; color: #666; margin-bottom: 30px; }}
            .navigation {{ 
                position: sticky; 
                top: 0; 
                background: white; 
                padding: 15px; 
                box-shadow: 0 2px 8px rgba(0,0,0,0.1); 
                z-index: 100;
                display: flex;
                justify-content: space-between;
                align-items: center;
                border-radius: 8px;
                margin-bottom: 20px;
            }}
            .nav-btn {{ 
                padding: 10px 20px; 
                background: #667eea; 
                color: white; 
                border: none; 
                border-radius: 4px; 
                cursor: pointer; 
                font-size: 16px;
                transition: background 0.3s;
            }}
            .nav-btn:hover:not(:disabled) {{ background: #764ba2; }}
            .nav-btn:disabled {{ 
                opacity: 0.5; 
                cursor: not-allowed; 
            }}
            .page-info {{
                color: #495057;
                font-weight: bold;
                font-size: 16px;
            }}
            .translation-page {{
                margin: 20px 0;
                border: 2px solid #e0e0e0;
                border-radius: 8px;
                overflow: hidden;
                background: white;
            }}
            .hint {{
                text-align: center;
                color: #666;
                font-size: 14px;
                margin: 20px 0;
                padding: 15px;
                background: #fff3cd;
                border-radius: 4px;
            }}
        </style>
        <script>
            let currentPage = 0;
            const totalPages = {len(results)};
            
            function showPage(pageIndex) {{
                if (pageIndex < 0 || pageIndex >= totalPages) return;
                
                // Hide all pages
                document.querySelectorAll('.translation-page').forEach(page => {{
                    page.style.display = 'none';
                }});
                
                // Show current page
                document.getElementById('page-' + pageIndex).style.display = 'block';
                
                // Update navigation
                currentPage = pageIndex;
                updateNavigation();
                
                // Scroll to top
                window.scrollTo({{ top: 0, behavior: 'smooth' }});
            }}
            
            function updateNavigation() {{
                document.getElementById('prev-btn').disabled = (currentPage === 0);
                document.getElementById('next-btn').disabled = (currentPage === totalPages - 1);
                
                const pageNum = {[r['page_num'] for r in results]}[currentPage];
                document.getElementById('page-info').textContent = 
                    `Page ${{pageNum}} (${{currentPage + 1}} of ${{totalPages}})`;
            }}
            
            function nextPage() {{
                showPage(currentPage + 1);
            }}
            
            function prevPage() {{
                showPage(currentPage - 1);
            }}
            
            // Keyboard navigation
            document.addEventListener('keydown', (e) => {{
                if (e.key === 'ArrowLeft') {{
                    prevPage();
                }} else if (e.key === 'ArrowRight') {{
                    nextPage();
                }}
            }});
            
            // Initialize on load
            window.onload = () => {{
                updateNavigation();
            }};
        </script>
    </head>
    <body>
        <h1>📄 arXiv Bilingual Translation</h1>
        <div class="timestamp">
            Paper: arXiv:{arxiv_id}<br>
            Translation: {source_lang} → {target_lang}<br>
            Generated: {time.strftime('%Y-%m-%d %H:%M:%S')}
        </div>
        
        <div class="navigation">
            <button id="prev-btn" class="nav-btn" onclick="prevPage()">◀ Previous</button>
            <span id="page-info" class="page-info"></span>
            <button id="next-btn" class="nav-btn" onclick="nextPage()">Next ▶</button>
        </div>
        
        <div class="hint">
            💡 提示：使用 ← → 方向鍵快速切換頁面
        </div>
        
        {pages_html}
        
        {terms_html}
    </body>
    </html>
    """
    
    return html

def debug_print(msg):
    """Print debug message if DEBUG_MODE is enabled"""
    if DEBUG_MODE:
        print(f"🔍 DEBUG: {msg}")

def extract_technical_terms(original, translation):
    """Extract technical terms"""
    terms = {}
    pattern = r'([\u4e00-\u9fff]+)\s*\(([a-zA-Z][a-zA-Z\s]+)\)'
    matches = re.findall(pattern, translation)
    for zh, en in matches:
        terms[en.strip()] = zh.strip()
    return terms

# ============================================
# Main Translation Flow (Text-Only Mode)
# ============================================
print("\n" + "#"*80)
print("# arXiv Bilingual Reader - Translation Started")
print("#"*80 + "\n")

# Download arXiv paper
pdf_path, total_pages = download_arxiv(ARXIV_ID)

# Calculate pages to translate
total_pages_to_translate = sum(end - start + 1 for start, end in SECTIONS.values())

print(f"📊 Translation plan:")
print(f"   • Mode: Text-only (fast and accurate)")
print(f"   • Total pages: {total_pages_to_translate}")
print(f"   • Display: {DISPLAY_MODE}")
print(f"   • Estimated time: ~{total_pages_to_translate * 23} seconds\n")

# Load text backend
print("🔄 Loading text backend...\n")
text_backend = TransformersBackend()
text_backend.load_model()
print("✅ Text backend ready\n")

all_terms = {}
results = []

# Translate all pages
print("\n" + "#"*80)
print("# Translation in Progress")
print("#"*80 + "\n")

with tqdm(total=total_pages_to_translate, desc="📖 Translating", unit="page") as pbar:
    for section_name, (start_page, end_page) in SECTIONS.items():
        for page_num in range(start_page, end_page + 1):
            pbar.set_description(f"📖 {section_name.capitalize()} - Page {page_num}")
            debug_print(f"Starting page {page_num} in section {section_name}")
            
            original_text = extract_text_from_page(pdf_path, page_num)
            debug_print(f"Extracted {len(original_text)} chars from page {page_num}")
            debug_print(f"Text preview: {original_text[:150]}...")
            
            if not original_text:
                debug_print(f"Page {page_num} has no text, skipping")
                print(f"⚠️  Page {page_num}: No text found, skipping...\n")
                pbar.update(1)
                continue
            
            start_time = time.time()
            debug_print(f"Calling translation API for page {page_num}...")
            result = text_backend.translate(
                original_text,
                source_lang=SOURCE_LANG,
                target_lang=TARGET_LANG
            )
            time_taken = time.time() - start_time
            translation = result['translation']
            debug_print(f"Translation received: {len(translation)} chars")
            debug_print(f"Translation preview: {translation[:150]}...")
            
            # Store result
            results.append({
                debug_print(f"Stored result for page {page_num}")
                'page_num': page_num,
                'section': section_name,
                'original': original_text,
                'translation': translation,
                'time': time_taken
            })
            
            # Extract terms
            if EXTRACT_TERMS:
                terms = extract_technical_terms(original_text, translation)
                all_terms.update(terms)
            
            pbar.update(1)

# Free backend
print("\n🗑️  Freeing text backend from GPU memory...")
del text_backend.model
del text_backend.tokenizer
del text_backend
gc.collect()
torch.cuda.empty_cache()
print("✅ Text backend freed\n")

# ============================================
# Display Results with Navigation
# ============================================
print("\n" + "#"*80)
print("# Translation Results - Interactive Preview")
print("#"*80 + "\n")

if len(results) > 1:
    print(f"📄 共 {len(results)} 頁翻譯結果")
    print(f"💡 向下滾動查看所有頁面\n")

# Display each page
for idx, result in enumerate(results):
    display_bilingual_rich([result], 0)

# Display technical terms glossary
if EXTRACT_TERMS and all_terms:
    print("\n" + "#"*80)
    print("# 📚 Technical Terms Glossary")
    print("#"*80 + "\n")
    
    print("="*80)
    print("專業術語對照表")
    print("="*80)
    
    for en, zh in sorted(all_terms.items()):
        print(f"{en:40} → {zh}")
    
    print("\n" + "="*80)
    print(f"Total terms extracted: {len(all_terms)}")
    print("="*80)

# ============================================
# Save Interactive HTML
# ============================================
if SAVE_TO_FILE:
    html_output = generate_interactive_html(results, ARXIV_ID, SOURCE_LANG, TARGET_LANG, all_terms)
    output_filename = f"translation_{ARXIV_ID}_{SOURCE_LANG}-{TARGET_LANG}.html"
    
    with open(output_filename, 'w', encoding='utf-8') as f:
        f.write(html_output)
    
    print(f"\n💾 Interactive HTML saved to: {output_filename}")
    print(f"\n✨ Features:")
    print(f"   • 左右箭頭按鈕切換頁面")
    print(f"   • 鍵盤 ← → 方向鍵快速導航")
    print(f"   • 固定導航欄顯示當前頁面")
    print(f"\n📥 使用下方的下載 cell 下載檔案\n")

print("\n" + "#"*80)
print("# ✅ Translation Complete!")
print("#"*80 + "\n")
print("💡 Summary:")
print(f"   • Translated {total_pages_to_translate} pages")
print(f"   • Sections: {', '.join(SECTIONS.keys())}")
print(f"   • Technical terms found: {len(all_terms)}")
print(f"   • Display mode: {DISPLAY_MODE}")
if SAVE_TO_FILE:
    print(f"   • Interactive HTML: {output_filename}")

## 📝 Notes & Tips

### 🎯 How to Use This Notebook

1. **First Time Setup**:
   - Run Cell 1-2 to setup environment (takes ~2-3 minutes)
   - Add your HuggingFace token to Colab Secrets

2. **Configure Your Paper** (Cell 3):
   - Change `ARXIV_ID` to your target paper
   - Adjust `SECTIONS` to translate specific parts
   - Mark pages with figures/tables in `IMAGE_MODE_PAGES`

3. **Run Translation** (Cell 4):
   - Scroll through bilingual results
   - Check technical terms glossary at the end

### ⚡ Performance Tips

- **Text Mode**: 20-25s/page - Use for pure text pages
- **Image Mode**: 40-50s/page - Use for pages with figures/tables
- **DPI Setting**: Lower DPI (72-96) is faster, higher (150) preserves details

### 🖼️ When to Use Image Mode?

Use `IMAGE_MODE_PAGES` for pages containing:
- Figures (charts, diagrams, architecture)
- Tables (results, comparisons)
- Complex layouts (multi-column, mixed text/images)

Image mode preserves the visual context and relationship between text and graphics.

### 📚 Bilingual Reading Benefits

- **Learn Technical Terms**: See how concepts are expressed in English
- **Verify Translation**: Check if key terms are translated correctly
- **Academic Writing**: Learn English academic writing patterns
- **Research Quality**: Ensure you understand the original meaning

### 🔧 Customization

**To translate different sections**:
```python
SECTIONS = {
    "introduction": (2, 3),
    "related_work": (4, 5),
    "conclusion": (15, 16),
}
```

**To disable bilingual mode** (translation only):
```python
BILINGUAL_MODE = False
```

**To translate to other languages**:
```python
TARGET_LANG = "ja"     # Japanese
TARGET_LANG = "ko"     # Korean
TARGET_LANG = "zh-CN"  # Simplified Chinese
```

---

## 🔗 Links

- 📦 **GitHub Repository**: [jimmyliao/trans-gemma](https://github.com/jimmyliao/trans-gemma)
- 🤗 **TranslateGemma Model**: [HuggingFace](https://huggingface.co/google/translategemma-4b-it)
- 📄 **Technical Report**: [arXiv:2601.09012](https://arxiv.org/abs/2601.09012)
- 🌐 **Google Blog**: [TranslateGemma Announcement](https://blog.google/innovation-and-ai/technology/developers-tools/translategemma/)

---

## 📄 License

MIT License - Free for commercial and non-commercial use

---

**有問題或建議？** 歡迎到 [GitHub Issues](https://github.com/jimmyliao/trans-gemma/issues) 提出！

## 📥 Download Translation

**下載翻譯結果到本地**

Execute the cell below to download the HTML file to your local machine.

In [ ]:
from google.colab import files
import os

# Find HTML translation files
html_files = [f for f in os.listdir('.') if f.startswith('translation_') and f.endswith('.html')]

if html_files:
    print(f"📄 找到 {len(html_files)} 個翻譯檔案：\n")
    
    for html_file in html_files:
        file_size = os.path.getsize(html_file) / 1024  # KB
        print(f"   • {html_file} ({file_size:.1f} KB)")
    
    # Download the most recent one
    html_file = html_files[0]
    print(f"\n📥 下載檔案：{html_file}")
    print("   檔案會出現在你的瀏覽器下載資料夾...\n")
    
    files.download(html_file)
    
    print("\n✅ 下載完成！")
    print("\n💡 接下來：")
    print("   1. 到瀏覽器的下載資料夾找到檔案")
    print("   2. 雙擊檔案用瀏覽器開啟")
    print("   3. 享受完美排版的雙語翻譯！")
else:
    print("❌ 找不到 HTML 檔案")
    print(f"\n當前目錄：{os.getcwd()}")
    print(f"\n所有檔案：")
    for f in sorted(os.listdir('.')):
        print(f"   • {f}")